In [1]:
ls -lah ../data/*

-rw-r--r-- 1 ilya lab 1.9G Jun 22 11:27 ../data/Undetermined_S0_R1_001.fastq.gz
-rw-r--r-- 1 ilya lab 2.0G Jun 22 11:27 ../data/Undetermined_S0_R2_001.fastq.gz

../data/ve01:
total 1.7G
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:41 .
drwxrwsr-x 16 ilya lab 4.0K Jun 22 11:37 ..
-rw-rw-r--  1 ilya lab 662K Jun 22 11:41 R1_fastqc.html
-rw-rw-r--  1 ilya lab 472K Jun 22 11:41 R1_fastqc.zip
-rw-rw-r--  1 ilya lab 855M Jun 22 11:37 R1.fastq.gz
-rw-rw-r--  1 ilya lab 663K Jun 22 11:41 R2_fastqc.html
-rw-rw-r--  1 ilya lab 467K Jun 22 11:41 R2_fastqc.zip
-rw-rw-r--  1 ilya lab 873M Jun 22 11:37 R2.fastq.gz

../data/ve02:
total 1.6G
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:43 .
drwxrwsr-x 16 ilya lab 4.0K Jun 22 11:37 ..
-rw-rw-r--  1 ilya lab 667K Jun 22 11:43 R1_fastqc.html
-rw-rw-r--  1 ilya lab 479K Jun 22 11:43 R1_fastqc.zip
-rw-rw-r--  1 ilya lab 767M Jun 22 11:37 R1.fastq.gz
-rw-rw-r--  1 ilya lab 669K Jun 22 11:43 R2_fastqc.html
-rw-rw-r--  1 ilya lab 475K Jun 22 11:43 R2_fastqc.zip
-rw-rw-r-- 

In [2]:
PATTERN="ve"
NPROC=128
base_dir="../data"

In [3]:
ls -lah ../data/

total 3.8G
drwxrwsr-x 16 ilya lab 4.0K Jun 22 11:37 .
drwxrwsr-x  6 ilya lab 4.0K Jun 22 11:20 ..
-rw-r--r--  1 ilya lab 1.9G Jun 22 11:27 Undetermined_S0_R1_001.fastq.gz
-rw-r--r--  1 ilya lab 2.0G Jun 22 11:27 Undetermined_S0_R2_001.fastq.gz
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:41 ve01
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:43 ve02
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve03
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve04
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve05
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve06
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve07
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve08
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve09
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve10
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve11
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve12
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve13
drwxrwsr-x  2 ilya lab 4.0K Jun 22 11:37 ve14


In [ ]:
ECOLI="../ref/MG1655"

read1="../data/read1.fifo"
read2="../data/read2.fifo"
mkfifo $read1
mkfifo $read2
    
for sample in $(ls $base_dir | grep "$PATTERN")
do
    sample_dir="$base_dir/$sample"
    result_dir="../results/$sample"
    
    mkdir -p $result_dir
    
    zcat "$sample_dir/R1.fastq.gz" > $read1 &
    zcat "$sample_dir/R2.fastq.gz" > $read2 &

    
    echo "Aligning $sample ..."
    # Align using bowtie2
    ~/bin/bowtie2/bowtie2 -p $NPROC --very-sensitive -x $ECOLI \
        -1 $read1 -2 $read2 \
        | samtools view -bhS - > "$result_dir/$sample.bam"
done

rm -rf ../data/*.fifo

[1] 2494977
[2] 2494978
Aligning ve01 ...
25190027 reads; of these:
  25190027 (100.00%) were paired; of these:
    1431071 (5.68%) aligned concordantly 0 times
    22884061 (90.85%) aligned concordantly exactly 1 time
    874895 (3.47%) aligned concordantly >1 times
    ----
    1431071 pairs aligned concordantly 0 times; of these:
      78199 (5.46%) aligned discordantly 1 time
    ----
    1352872 pairs aligned 0 times concordantly or discordantly; of these:
      2705744 mates make up the pairs; of these:
        2596900 (95.98%) aligned 0 times
        97137 (3.59%) aligned exactly 1 time
        11707 (0.43%) aligned >1 times
94.85% overall alignment rate
[1]-  Done                    zcat "$sample_dir/R1.fastq.gz" > $read1
[2]+  Done                    zcat "$sample_dir/R2.fastq.gz" > $read2
[1] 2504530
[2] 2504531
Aligning ve02 ...
22479140 reads; of these:
  22479140 (100.00%) were paired; of these:
    2746847 (12.22%) aligned concordantly 0 times
    19059495 (84.79%) aligne

In [8]:
PATTERN="ve"

for sample in $(ls $base_dir | grep "$PATTERN")
do
    result_dir="../results/$sample"
    samtools sort -@ $NPROC "$result_dir/$sample.bam" -o "$result_dir/${sample}_sorted.bam" &
done

[1] 655810
[2] 655811
[3] 655812
[4] 655813
[5] 655814
[6] 655815
[7] 655816
[8] 655817
[9] 655818
[10] 655819
[11] 655820
[12] 655821
[13] 655822
[14] 655823


In [9]:
ls -lah ../results/*

[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
[bam_sort_core] merging from 0 files and 128 in-memory blocks...
../results/ve01:
total 345M
drwxrwsr-x  2 ilya lab 4.0K May 19 14:44 .
drwxrwsr-x 16 ilya 

In [10]:
for sample in $(ls $base_dir | grep "$PATTERN")
do
    result_dir="../results/$sample"
    samtools index -@ $NPROC "$result_dir/${sample}_sorted.bam" &
done

[1] 661768
[2] 661769
[3] 661770
[4] 661771
[5] 661772
[6] 661773
[7] 661774
[8] 661775
[9] 661776
[10] 661777
[11] 661778
[12] 661779
[13] 661780
[14] 661781


In [11]:
ls -lah ../results/*

../results/ve01:
total 346M
drwxrwsr-x  2 ilya lab 4.0K May 19 14:44 .
drwxrwsr-x 16 ilya lab 4.0K May 19 14:40 ..
-rw-rw-r--  1 ilya lab 221M May 19 14:32 ve01.bam
-rw-rw-r--  1 ilya lab 125M May 19 14:44 ve01_sorted.bam
-rw-rw-r--  1 ilya lab  12K May 19 14:44 ve01_sorted.bam.bai

../results/ve02:
total 324M
drwxrwsr-x  2 ilya lab 4.0K May 19 14:44 .
drwxrwsr-x 16 ilya lab 4.0K May 19 14:40 ..
-rw-rw-r--  1 ilya lab 206M May 19 14:32 ve02.bam
-rw-rw-r--  1 ilya lab 118M May 19 14:44 ve02_sorted.bam
-rw-rw-r--  1 ilya lab  11K May 19 14:44 ve02_sorted.bam.bai

../results/ve03:
total 385M
drwxrwsr-x  2 ilya lab 4.0K May 19 14:44 .
drwxrwsr-x 16 ilya lab 4.0K May 19 14:40 ..
-rw-rw-r--  1 ilya lab 241M May 19 14:33 ve03.bam
-rw-rw-r--  1 ilya lab 144M May 19 14:44 ve03_sorted.bam
-rw-rw-r--  1 ilya lab  11K May 19 14:44 ve03_sorted.bam.bai

../results/ve04:
total 278M
drwxrwsr-x  2 ilya lab 4.0K May 19 14:44 .
drwxrwsr-x 16 ilya lab 4.0K May 19 14:40 ..
-rw-rw-r--  1 ilya lab 176M May 1